<a href="https://www.kaggle.com/code/nigamshitij/parse-book-reviews-using-duckduckgo?scriptVersionId=195384732" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-review-samples/goodreads_fantasy.csv
/kaggle/input/book-review-samples/goodreads_all_genres.csv
/kaggle/input/book-review-samples/Darth_Plagueis_reviews(4).csv
/kaggle/input/book-review-samples/goodreads_all_genres_final.csv
/kaggle/input/book-review-samples/Dune_Book_1_reviews.csv
/kaggle/input/book-review-samples/Dune_Book_1_reviews(4).csv
/kaggle/input/book-review-samples/goodreads_genres_complete.csv


In [2]:
pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import random
import re
import html
from duckduckgo_search import DDGS
import glob
import csv
from tqdm import tqdm

In [4]:
# boilerplate removal

def remove_boilerplate(text):
    boilerplate = ['cookie policy', 'privacy policy', 'terms of service', 'all rights reserved', '\n']
    for phrase in boilerplate:
        text = re.sub(r'(?i)' + re.escape(phrase) + r'.*', '', text)
    return text

In [5]:
# comment truncation (20% threshold default)

def truncate_at_comments(review_text, threshold_percentage=20, min_review_length=100):
    comment_identifiers = [
        'Comments', 'Leave a comment', 'Reader comments', 
        'What do you think?', 'Join the discussion', 'Add a comment',
        'Post a comment', 'Write a comment', 'Show comments'
    ]
    
    lower_text = review_text.lower()
    text_length = len(lower_text)
    threshold = max(int(text_length * (threshold_percentage / 100)), min_review_length)

    # Check for comment identifiers
    for identifier in comment_identifiers:
        index = lower_text.find(identifier.lower())
        if index != -1 and index > threshold:
            return review_text[:index].strip()
    
    # If no identifiers found, try to detect comment-like structures
    paragraphs = review_text.split('\n\n')
    filtered_paragraphs = []
    
    for paragraph in paragraphs:
        # Skip short paragraphs that might be comments
        if len(paragraph) < 50:
            continue
        
        # Skip paragraphs that start with common comment patterns
        if re.match(r'^(Posted by|From|User|Anonymous|[\d/]+:)', paragraph.strip()):
            continue
        
        filtered_paragraphs.append(paragraph)
    
    # If we've removed some paragraphs, join the remaining ones
    if len(filtered_paragraphs) < len(paragraphs):
        return '\n\n'.join(filtered_paragraphs).strip()
    
    # If we haven't removed any paragraphs, return at least the first part of the text
    return review_text[:max(threshold, len(review_text) // 2)].strip()

In [6]:
# remove comment classes

def remove_comments(review_soup):
    # Common class names for comment sections
    comment_classes = ['comment', 'comments-list', 'comments-area', 'comments', 'comment-section', 'user-comments', 'disqus_thread']
    
    for class_name in comment_classes:
        comment_section = review_soup.find('div', class_=class_name)
        if comment_section:
            comment_section.decompose()  # This removes the element from the soup
    
    return review_soup

In [7]:
# paragraph scoring for review content

def score_paragraph(paragraph):
    review_keywords = ['review', 'book', 'read', 'author', 'story', 'character', 'plot', 'recommend']
    return sum(keyword in paragraph.lower() for keyword in review_keywords)

In [8]:
# add retries with timeouts selectively

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib3.exceptions import ReadTimeoutError
from bs4 import BeautifulSoup
import time
import random

def create_session_with_retries():
    session = requests.Session()
    retries = Retry(total=5, 
                    backoff_factor=1, 
                    status_forcelist=[429, 500, 502, 503, 504],
                    allowed_methods=["HEAD", "GET", "OPTIONS"])
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [9]:
exceptions = ('google.', 'goodreads.com', 'amazon.', 'reddit.com', 'thestorygraph', 'youtube','.tv', 'barnesandnoble.com','wikipedia.','quora.com','sparknotes.com', 'grimdarkmagazine.', 'washingtonpost.')

def process_review_url(review_url, headers):
    reviews = []
    # Skip known non-review sites
    # if any(site in review_url for site in ['google.com', 'wikipedia.org']):
    if any(site in review_url.lower() for site in exceptions):
        return reviews
    
    session = create_session_with_retries()
    
    try:
        # Fetch the review page
        # review_response = requests.get(review_url, headers=headers, timeout=10)
        review_response = session.get(review_url, headers=headers, timeout=10)  # Increased timeout to 20 seconds
        review_soup = BeautifulSoup(review_response.text, 'html.parser')
        
        # Remove Comments
        review_soup = remove_comments(review_soup)
        
        # Review elements
        review_elements = review_soup.find_all(['main','p'])
        scored_paragraphs = [(elem, score_paragraph(elem.text)) for elem in review_elements]
        
        # Extract review text
        review_paragraphs = [elem.text for elem, score in sorted(scored_paragraphs, key=lambda x: x[1], reverse=True)[:3]]
        
        # Search for relevant text
        review_text = ' '.join(review_paragraphs)
        
        # Remove boilerplate
        review_text = review_text.replace('\n', ' ')
        review_text = remove_boilerplate(review_text)
        
        # Truncate at Comments
        review_text = truncate_at_comments(review_text)
        
        review_date = 'Unknown'
        
        reviews.append({
            'review_text': review_text[:5000],  # Limit to first 5000 characters
            'review_date': review_date,
            'review_website': review_url
        })
        
    except ReadTimeoutError as e:
        print(f"Read timeout error for {review_url}: {str(e)}")
    except requests.exceptions.RequestException as e:
        print(f"Error processing {review_url}: {str(e)}")
    except Exception as e:
        print(f"Unexpected error processing {review_url}: {str(e)}")
    
    # Be polite to servers
    time.sleep(random.uniform(1, 3))
    
    return reviews

In [10]:
# search_book_review test

def search_book_reviews(book_name, author):
    # Combine book name and author for search query
    search_query = f"{book_name} {author} book review"
    
    # print(f"Searching for: {search_query}")
    
    reviews = []
    
    # Define headers
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    # Use DDGS for searching
    with DDGS() as ddgs:
        results = ddgs.text(search_query, max_results=10)  # Adjust max_results as needed
        
        for result in results:
            review_url = result['href']
            reviews.extend(process_review_url(review_url, headers))
    
    # Create DataFrame
    df = pd.DataFrame(reviews)
    
    return df

In [11]:
# # Example usage
# book_name = "Darth Plagueis"
# author = "James Luceno"

# df = search_book_reviews(book_name, author)

# # Save to CSV
# df.to_csv(f"{book_name.replace(' ', '_')}_reviews.csv", index=False)
# print(f"Reviews saved to {book_name.replace(' ', '_')}_reviews.csv")

# df.head()

In [12]:
# input_csv = '/kaggle/input/book-review-samples/goodreads_all_genres_final.csv'
# output_csv = 'book_reviews.csv'

input_csv = '/kaggle/input/book-review-samples/goodreads_all_genres_final.csv'
output_directory = '/kaggle/working/'
output_csv = 'all_book_reviews.csv'

df = pd.read_csv(input_csv)
df_unique = df.drop_duplicates(subset=['Title', 'Authors'], keep='first')
num_duplicates = len(df) - len(df_unique)
num_total = len(df)
print(f"Removed {num_duplicates} duplicate entries out of {num_total}")

Removed 0 duplicate entries out of 35597


In [13]:
def process_book_list_old(input_csv, output_csv):
    # Read the input CSV
    df = pd.read_csv(input_csv)
    
    # remove duplicates
    df_unique = df.drop_duplicates(subset=['Title', 'Authors'], keep='first')
    
    # Print information about removed duplicates
    num_duplicates = len(df) - len(df_unique)
    num_total = len(df)
    print(f"Removed {num_duplicates} duplicate entries out of {num_total}")
    
    all_reviews = []
    
    # Iterate through each row in the dataframe
    for index, row in tqdm(df_unique.iterrows(), total=df_unique.shape[0], desc="Processing books"):
        title = row['Title']
        authors = row['Authors']
        
        try:
            # Get reviews for this book
            reviews_df = search_book_reviews(title, authors)
            
            # Add book information to each review
            reviews_df['Title'] = title
            reviews_df['Authors'] = authors
            reviews_df['Avg Ratings'] = row['Avg Ratings']
            reviews_df['Rating'] = row['Rating']
            reviews_df['Published_year'] = row['Published_year']
            
            all_reviews.append(reviews_df)
        
        except Exception as e:
            print(f"Error processing {title} by {authors}: {str(e)}")
    
    # Combine all reviews into a single dataframe
    if all_reviews:
        final_df = pd.concat(all_reviews, ignore_index=True)
        
        # Save to CSV
        final_df.to_csv(output_csv, index=False, quoting=csv.QUOTE_ALL)
        print(f"Reviews saved to {output_csv}")
    else:
        print("No reviews were collected.")

In [14]:
def process_book_list_old_2(input_csv, output_csv):
    # Read the input CSV
    df = pd.read_csv(input_csv)
    
    # remove duplicates
    df_unique = df.drop_duplicates(subset=['Title', 'Authors'], keep='first')
    
    # Print information about removed duplicates
    num_duplicates = len(df) - len(df_unique)
    num_total = len(df)
    print(f"Removed {num_duplicates} duplicate entries out of {num_total}")
    
    all_reviews = []
    batch_size = 10
    batch_count = 0
    
    # Iterate through each row in the dataframe
    for index, row in tqdm(df_unique.iterrows(), total=df_unique.shape[0], desc="Processing books"):
        title = row['Title']
        authors = row['Authors']
        
        try:
            # Get reviews for this book
            reviews_df = search_book_reviews(title, authors)
            
            # Add book information to each review
            reviews_df['Title'] = title
            reviews_df['Authors'] = authors
            reviews_df['Avg Ratings'] = row['Avg Ratings']
            reviews_df['Rating'] = row['Rating']
            reviews_df['Published_year'] = row['Published_year']
            
            all_reviews.append(reviews_df)
        
        except Exception as e:
            print(f"Error processing {title} by {authors}: {str(e)}")
        
        # Save batch to CSV every 10 books
        if len(all_reviews) == batch_size:
            batch_df = pd.concat(all_reviews, ignore_index=True)
            batch_filename = f"/kaggle/working/batch_{batch_count}.csv"
            batch_df.to_csv(
                batch_filename, 
                index=False, 
                quoting=csv.QUOTE_ALL,
                escapechar='\\',
                doublequote=False
            )
            print(f"Batch {batch_count} saved to {batch_filename}")
            all_reviews = []
            batch_count += 1
    
    # Save any remaining reviews
    if all_reviews:
        batch_df = pd.concat(all_reviews, ignore_index=True)
        batch_filename = f"/kaggle/working/batch_{batch_count}.csv"
        batch_df.to_csv(batch_filename, index=False, quoting=csv.QUOTE_ALL)
        print(f"Final batch saved to {batch_filename}")
    
    # Combine all batches
    all_files = glob.glob("/kaggle/working/batch_*.csv")
    combined_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    
    # Save combined results
    combined_df.to_csv(
        output_csv, 
        index=False, 
        quoting=csv.QUOTE_ALL,
        escapechar='\\',
        doublequote=False
    )
    print(f"All reviews combined and saved to {output_csv}")
    
    # Optionally, remove batch files
    for f in all_files:
        os.remove(f)
    print("Batch files removed")

In [15]:
def process_book_list(input_csv, output_csv):
    # Read the input CSV
    df = pd.read_csv(input_csv)
    
    # Remove duplicates
    df_unique = df.drop_duplicates(subset=['Title', 'Authors'], keep='first')
    
    # Print information about removed duplicates
    num_duplicates = len(df) - len(df_unique)
    num_total = len(df)
    print(f"Removed {num_duplicates} duplicate entries out of {num_total}")
    
    # Find the last processed batch
    existing_batches = glob.glob("/kaggle/working/batch_*.csv")
    if existing_batches:
        last_batch = max(existing_batches, key=os.path.getctime)
        last_batch_number = int(last_batch.split('_')[1].split('.')[0])
        start_index = (last_batch_number + 1) * 10
    else:
        start_index = 0
    
    print(f"Resuming processing from index {start_index}")
    
    batch_size = 10
    batch_count = start_index // 10
    
    # Iterate through each row in the dataframe, starting from start_index
    for index in tqdm(range(start_index, len(df_unique)), total=len(df_unique)-start_index, desc="Processing books"):
        row = df_unique.iloc[index]
        title = row['Title']
        authors = row['Authors']
        
        try:
            # Get reviews for this book
            reviews_df = search_book_reviews(title, authors)
            
            # Add book information to each review
            reviews_df['Title'] = title
            reviews_df['Authors'] = authors
            reviews_df['Avg Ratings'] = row['Avg Ratings']
            reviews_df['Rating'] = row['Rating']
            reviews_df['Published_year'] = row['Published_year']
            
            # Save batch to CSV every 10 books
            if (index + 1) % batch_size == 0 or index == len(df_unique) - 1:
                batch_filename = f"/kaggle/working/batch_{batch_count}.csv"
                reviews_df.to_csv(batch_filename, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', doublequote=False)
                print(f"Batch {batch_count} saved to {batch_filename}")
                batch_count += 1
        
        except Exception as e:
            print(f"Error processing {title} by {authors}: {str(e)}")
    
    # Combine all batches
    all_files = glob.glob("/kaggle/working/batch_*.csv")
    combined_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    
    # Save combined results
    combined_df.to_csv(output_csv, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', doublequote=False)
    print(f"All reviews combined and saved to {output_csv}")
    
    # Optionally, remove batch files
    for f in all_files:
        os.remove(f)
    print("Batch files removed")

In [ ]:
# Usage
process_book_list(input_csv, output_csv)

Removed 0 duplicate entries out of 35597
Resuming processing from index 1560


Processing books:   0%|          | 10/34037 [01:47<108:19:53, 11.46s/it]

Batch 156 saved to /kaggle/working/batch_156.csv


Processing books:   0%|          | 12/34037 [02:13<121:12:57, 12.83s/it]

Error processing https://www.kirkusreviews.com/book-reviews/kim-mclarin/james-baldwins-another-country/: HTTPSConnectionPool(host='www.kirkusreviews.com', port=443): Max retries exceeded with url: /book-reviews/kim-mclarin/james-baldwins-another-country/ (Caused by ReadTimeoutError("HTTPSConnectionPool(host='www.kirkusreviews.com', port=443): Read timed out. (read timeout=10)"))


...

Processing books:   8%|▊         | 2674/34037 [11:56:49<206:27:28, 23.70s/it]

Error processing https://www.kirandellimore.com/book-review-midnights-children-salman-rushdie/: HTTPSConnectionPool(host='www.kirandellimore.com', port=443): Max retries exceeded with url: /book-review-midnights-children-salman-rushdie/ (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Processing books:   8%|▊         | 2676/34037 [11:58:05<257:26:27, 29.55s/it]

Error processing https://farflungreaders.com/book-review-crying-in-h-mart-by-michelle-zauner/: HTTPSConnectionPool(host='farflungreaders.com', port=443): Max retries exceeded with url: /book-review-crying-in-h-mart-by-michelle-zauner/ (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Processing books:   8%|▊         | 2677/34037 [11:59:06<339:54:36, 39.02s/it]

In [ ]:
print("2707 batch 361")

In [ ]:
print("test2")